# Kalman Filter Models
> Models that use Kalman filters that can be used for imputation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from fastcore.basics import store_attr
import numpy as np

## Introduction

[TODO] add proper introduction here

The models uses a latent state variable $x$ that is modelled over time, to impute gaps in $y$

### Equations

The equations of the model are:

$$\begin{align} p(x_t | x_{t-1}) & = \mathcal{N}(x_t, Ax_{t-1}, Q) \\
p(y_t | x_t) & = \mathcal{N}(Hx_t, T) \end{align}$$

The Kalman filter has 3 steps:

- filter (updating the state at time t with observations till time t-1)
- update (update the state at time t using the observation at time t)
- smooth (update the state using the observations at time t+1)

In case of missing data the update step is skipped.

After smoothing the whole dataset the missing data ($y_t$) can be imputed from the state ($x_t$) using this formula:
$$p(y_t) = \mathcal{N}(Hx_x, R + HP^s_tH)$$

## Kalman Filter Model

general kalman filter model implemented used `pykalman`


In [ ]:
#| export
import pykalman
from numpy.ma import MaskedArray
import numpy as np
from fastcore.meta import delegates
from typing import Collection
from collections import namedtuple

#### PyKalman

examples from [pykalman lib](https://pykalman.github.io)

In [ ]:
kf = pykalman.KalmanFilter(transition_matrices = [[1, 1], [0, 1]], observation_matrices = [[0.1, 0.5], [-0.3, 0.0]])
measurements = np.asarray([[1,0], [0,0], [0,1]])
(smoothed_state_means, smoothed_state_covariances) = kf.smooth(measurements)

In [ ]:
kf.em(measurements)

<pykalman.standard.KalmanFilter>

In [ ]:
smoothed_state_means

array([[-0.10923868,  0.0935127 ],
       [-0.23121289, -0.07957144],
       [-0.5533711 , -0.0415223 ]])

In [ ]:
smoothed_state_covariances

array([[[ 0.83148067, -0.12300405],
        [-0.12300405,  0.53081415]],

       [[ 1.60960449,  0.01009906],
        [ 0.01009906,  0.80412661]],

       [[ 2.87663094,  0.45474213],
        [ 0.45474213,  1.27365905]]])

In [ ]:
smoothed_state_covariances.shape

(3, 2, 2)

In [ ]:
#| export
# this is just a duplicate of NormParam, but with a better name
ListNormal = namedtuple('ListNormal', ['mean', 'cov'])

support is not limited for models matrices that don't change over time

In [ ]:
#| export
class KalmanModel():
    em_vars = 'all'
    "Base Model for Kalman filter that wraps `pykalman.KalmanFilter`"
    @delegates(pykalman.KalmanFilter)
    def __init__(self,
                 data: MaskedArray, # numpy array of observations
                 **kwargs):
        self.data = data
        self.n_obs = data.shape[0]
        self.n_features = data.reshape(self.n_obs, -1).shape[1]
        
        self.model = pykalman.KalmanFilter(**kwargs)
    
    def fit(self, n_iter: int=5, em_vars: Collection[str]=None):
        "fit parameters using EM and calc state using smoother"
        em_vars = em_vars if em_vars is not None else self.em_vars # subclass can override this
        self.model.em(self.data, n_iter=n_iter, em_vars=em_vars)
        mean, cov = self.model.smooth(self.data)
        self.state = ListNormal(mean, cov)
        return self
    
    def _obs_from_state(self, state_mean, state_cov):
        mean = self.model.observation_matrices @ state_mean
        cov = self.model.observation_matrices @ state_cov @ self.model.observation_matrices + self.model.observation_covariance
        return mean, cov
    def predict(self,
                times: np.ndarray # times for predictions (indices of the training data)
               ):
        "Predicts observed varibles values at the given times"
        assert times.max() <= self.n_obs
        means = np.empty((times.shape[0], self.n_features,))
        covs = np.empty((times.shape[0], self.n_features, self.n_features,)) 
        for i, t in enumerate(times):
            mean, cov = self._obs_from_state(self.state.mean[t], self.state.cov[t])
            means[i] = mean
            covs[i] = cov
        return ListNormal(means, covs)
    

In [ ]:
X = np.hstack([np.arange(0,3), np.arange(3, 0, -1)])

In [ ]:
X

array([0, 1, 2, 3, 2, 1])

In [ ]:
k = KalmanModel(X)

In [ ]:
k.fit(10)

<__main__.KalmanModel>

In [ ]:
T = np.arange(0,X.shape[0])

In [ ]:
T

array([0, 1, 2, 3, 4, 5])

In [ ]:
k.predict(T)

ListNormal(mean=array([[-0.07657965],
       [ 0.99577849],
       [ 1.87172949],
       [ 2.59099059],
       [ 1.88745786],
       [ 1.15831217]]), cov=array([[[0.13676111]],

       [[0.20354109]],

       [[0.20417844]],

       [[0.20418473]],

       [[0.20420701]],

       [[0.20653508]]]))

## Local Level Model

Local level models is a model that uses Kalman filter, where the design matrix ($A$) and the Transition matrix ($H$) are identity matrix. This means that the state of model is equal to the observations and the changes in the state are only from the process noise.

Those are reasonable 

In [ ]:
class LocalLevel(KalmanModel):
    "Local level model using a kalman filter"
    em_vars = []

**Warning** this implementation of the EM algorithm may actually result in matrices that aren't correct for multivariate variables

## Export 

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()